# Capstone Project - The Battle of Neighborhoods (Week 1)

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# 1- Introduction: Business Problem <a name="introduction"></a>

#### Where you discuss the business problem and who would be interested in this project.

**Description :** In this project we will try to find an optimal location for a restaurant specialized in Dominican Food, the stakeholders owns two locations in Patterson NJ(USA) already, but she is interested in expanding her business to another state, in this case Georgia.

**Background:** The stakeholder knows there is a lot of people traveling through Georgia, due to Atlanta having an international airport, and being a busy city, she wants to make sure to get an ideal place, not too far from the airport, and where this type of restaurant is not common.  

We will use our Data Science skills to find the ideal place that meets with these criteria, close to the airport, and in an area where restaurants are common, but here are not Dominican Restaurants yet, is OK any other type of Caribbean restaurants.

# 2- Data <a name="data"></a>

We were given 3 specific cities closed to the Airport to look into (**Atlanta, Decatur and Kennesaw**), and then select the best place for our Dominican Restaurant.  

- First I used Foresquare API to collect the venues within 10 miles of each city center.  
- Second I filtered the results and collected just the restaurants found within 10 miles of each city center.

- Third I performed a search to make sure there were no other Dominican restaurants in the area.

- Finally, I generated a plot with each restaurant found in each city center, to have a visual of how close are these restaurants to each other, this will give us an overview of the data that we're working with to find the best city to place our Dominican Restaurant.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

In [2]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


def json_to_dataframe(items):
    dataframe = json_normalize(items) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

    return dataframe_filtered

**Fourtsquare Credentials**

In [3]:
# Fourtsquare Credentials
CLIENT_ID = '0SNZYNIMMXS2QECV0POPXLWQNXYC4DIPSFARJEYD1TBSCLTA' # your Foursquare ID
CLIENT_SECRET = 'SEZBRZ1VRC0PKCUTBSZUXTS021LBV5TZLIBU2UJX1YW5KM1H' # your Foursquare Secret
VERSION = '20180604'
RADIUS = 16093.4  # 10 miles
LIMIT = 100
# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

### Exploring our 3 locations

**Atlanta Data Collection**

In [4]:
# Atlanta Coordinates
atlanta = {'lat':33.749, 'lng':-84.388}

# Atlanta URL to perform venew search
url_atlanta = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, atlanta['lat'], atlanta['lng'], VERSION, RADIUS, LIMIT)
results_atlanta = requests.get(url_atlanta).json()


# Collecting venue items into a DataFrame
items_atlanta = results_atlanta['response']['groups'][0]['items']
df_atlanta = json_to_dataframe(items_atlanta)

'There are {} venues within 10 miles of Atlanta center.'.format(len(results_atlanta['response']['groups'][0]['items']))

'There are 100 venues within 10 miles of Atlanta center.'

In [5]:
# Collecting restaurants in our Dataframe
df_atl_rest = df_atlanta[df_atlanta["categories"].str.contains('Restaurant')]
df_atl_rest.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Nick's Food To Go,Greek Restaurant,240 MLK Dr SE,US,Atlanta,United States,at Hill St,781,"[240 MLK Dr SE (at Hill St), Atlanta, GA 30312...","[{'label': 'display', 'lat': 33.7474921065436,...",33.747492,-84.379749,NaN,30312,GA,4abbf3d9f964a520a28520e3
1,Aviva by Kameel,Mediterranean Restaurant,225 Peachtree St NE,US,Atlanta,United States,In the Mall at Peachtree Center,1297,[225 Peachtree St NE (In the Mall at Peachtree...,"[{'label': 'display', 'lat': 33.7605969735783,...",33.760597,-84.386548,NaN,30303,GA,50731cc9e4b06489e10a354d
25,Atlanta Breakfast Club,American Restaurant,249 Ivan Allen Jr Blvd NW,US,Atlanta,United States,at Venable St NW,1888,"[249 Ivan Allen Jr Blvd NW (at Venable St NW),...","[{'label': 'display', 'lat': 33.76479891528359...",33.764799,-84.395432,NaN,30313,GA,53298b6b498e04fb57694aae
31,Superica,Mexican Restaurant,99 Krog St NE,US,Atlanta,United States,NaN,2367,"[99 Krog St NE, Atlanta, GA 30307, United States]","[{'label': 'display', 'lat': 33.75700282446537...",33.757003,-84.364299,NaN,30307,GA,54deafbf498eab5eadfd5319
32,Poor Calvin's,Thai Restaurant,510 Piedmont Ave NE,US,Atlanta,United States,Renaissance Pkwy. NE,2223,"[510 Piedmont Ave NE (Renaissance Pkwy. NE), A...","[{'label': 'display', 'lat': 33.7684, 'lng': -...",33.768400,-84.382260,NaN,30308,GA,50a570e38aca7f0bfeca898c


In [6]:
# Amount of restaurants found within 10 miles of Atlanta center.
df_atl_rest["categories"].count()

23

In [25]:
# Amount of restaurants with the word Dominican in their name found within 10 miles of Atlanta center.
df_atl_rest["categories"][df_atl_rest["categories"].str.contains('Dominican|Dominicano')].count()

0

In [8]:
atlanta_map = folium.Map(location=[atlanta['lat'], atlanta['lng']], zoom_start=13)

# Add restaurants to the map
for lat, lng, label in zip(df_atl_rest.lat, df_atl_rest.lng, df_atl_rest.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='yellow',
        fill_opacity=0.6
        ).add_to(atlanta_map)

# display map
atlanta_map

**Decatur Data Collection**

In [9]:
decatur = {'lat':33.774, 'lng':-84.296}

# Decatur URL to perform venew search
url_decatur = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, decatur['lat'], decatur['lng'], VERSION, RADIUS, LIMIT)
results_decatur = requests.get(url_decatur).json()

# Collecting venue items into a DataFrame
items_decatur = results_decatur['response']['groups'][0]['items']
df_decatur = json_to_dataframe(items_decatur)

'There are {} venues within 10 miles of Decatur center.'.format(len(results_decatur['response']['groups'][0]['items']))

'There are 100 venues within 10 miles of Decatur center.'

In [10]:
# Collecting restaurants in our Dataframe
df_dec_rest = df_decatur[df_decatur["categories"].str.contains('Restaurant')]
df_dec_rest.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
1,The Iberian Pig,Spanish Restaurant,121 Sycamore St,US,Decatur,United States,NaN,42,"[121 Sycamore St, Decatur, GA 30030, United St...","[{'label': 'display', 'lat': 33.77438, 'lng': ...",33.774380,-84.296000,NaN,30030,GA,4aa2b2f8f964a520164220e3
5,No. 246,Italian Restaurant,129 E Ponce de Leon Ave,US,Decatur,United States,btwn Church St & E Court Sq,181,[129 E Ponce de Leon Ave (btwn Church St & E C...,"[{'label': 'display', 'lat': 33.77553761696676...",33.775538,-84.295349,NaN,30030,GA,4daf0acab3dcd1a46a0b5e57
10,The Pinewood,American Restaurant,254 W Ponce de Leon Ave,US,Decatur,United States,at Ponce de Leon Pl.,401,[254 W Ponce de Leon Ave (at Ponce de Leon Pl....,"[{'label': 'display', 'lat': 33.77548088894041...",33.775481,-84.299958,Downtown Decatur,30030,GA,4fa5cb03e4b094b946a36158
26,Madras Mantra,South Indian Restaurant,2179 Lawrenceville Hwy Ste A,US,Decatur,United States,NaN,4673,"[2179 Lawrenceville Hwy Ste A, Decatur, GA 300...","[{'label': 'display', 'lat': 33.81060798417284...",33.810608,-84.271270,NaN,30033,GA,55679b78498efa7eb6a22221
33,Surin of Thailand,Thai Restaurant,810 N Highland Ave NE,US,Atlanta,United States,NaN,5252,"[810 N Highland Ave NE, Atlanta, GA 30306, Uni...","[{'label': 'display', 'lat': 33.7765778, 'lng'...",33.776578,-84.352685,NaN,30306,GA,49dff3f8f964a5202e611fe3


In [11]:
# Amount of restaurants found within 10 miles of Atlanta center.
df_dec_rest["categories"].count()

19

In [24]:
# Amount of restaurants with the word Dominican in their name found within 10 miles of Atlanta center.
df_dec_rest["categories"][df_dec_rest["categories"].str.contains('Dominican|Dominicano')].count()

0

In [13]:
decatur_map = folium.Map(location=[decatur['lat'], decatur['lng']], zoom_start=13)

# Add restaurants to the map
for lat, lng, label in zip(df_dec_rest.lat, df_dec_rest.lng, df_dec_rest.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='yellow',
        fill_opacity=0.6
        ).add_to(decatur_map)

# display map
decatur_map

**Kennesaw Data Collection**

In [16]:
kennesaw = {'lat':34.023, 'lng':-84.615}

# Atlanta URL to perform venew search
url_kennesaw = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, kennesaw['lat'], kennesaw['lng'], VERSION, RADIUS, LIMIT)
results_kennesaw = requests.get(url_kennesaw).json()


# Collecting venue items into a DataFrame
items_kennesaw = results_kennesaw['response']['groups'][0]['items']
df_kennesaw = json_to_dataframe(items_kennesaw)

'There are {} venues within 10 miles of Kennesaw center.'.format(len(results_kennesaw['response']['groups'][0]['items']))

'There are 100 venues within 10 miles of Kennesaw center.'

In [17]:
# Collecting restaurants in our Dataframe
df_ken_rest = df_kennesaw[df_kennesaw["categories"].str.contains('Restaurant')]
df_ken_rest.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
13,Capers,American Restaurant,1659-1705 Old U.S. 41 NW,US,Acworth-Kennesaw,United States,NaN,3555,"[1659-1705 Old U.S. 41 NW, Acworth-Kennesaw, G...","[{'label': 'display', 'lat': 33.99405415284648...",33.994054,-84.598705,30152,GA,4b857420f964a520f75e31e3
17,Carrabba's Italian Grill,Italian Restaurant,1160 Ernest W Barrett Pkwy NW,US,Kennesaw,United States,NaN,3785,"[1160 Ernest W Barrett Pkwy NW, Kennesaw, GA 3...","[{'label': 'display', 'lat': 34.00108100441834...",34.001081,-84.583639,30144,GA,4b2abebdf964a520bcae24e3
18,Marlow's Tavern,New American Restaurant,"745 Chastain Road NW, Suite 1160",US,Kennesaw,United States,at George Busbee Pkwy,4274,"[745 Chastain Road NW, Suite 1160 (at George B...","[{'label': 'display', 'lat': 34.0348057, 'lng'...",34.034806,-84.570914,30144,GA,4a7cbf12f964a52073ed1fe3
21,Chick-fil-A,Fast Food Restaurant,830 Ernest W Barrett Pkwy Nw,US,Kennesaw,United States,NaN,4182,"[830 Ernest W Barrett Pkwy Nw, Kennesaw, GA 30...","[{'label': 'display', 'lat': 34.0070494, 'lng'...",34.007049,-84.573964,30144,GA,4b4bd76af964a52049a926e3
24,Copeland's of New Orleans,Cajun / Creole Restaurant,1142 Ernest W Barrett Pkwy NW,US,Kennesaw,United States,at Roberts Blvd NW,3811,[1142 Ernest W Barrett Pkwy NW (at Roberts Blv...,"[{'label': 'display', 'lat': 34.00161679569724...",34.001617,-84.582746,30144,GA,4ab0469ff964a520ca6620e3


In [18]:
# Amount of restaurants found within 10 miles of Atlanta center.
df_ken_rest["categories"].count()

28

In [23]:
# Amount of restaurants with the word Dominican in their name found within 10 miles of Atlanta center.
df_ken_rest["categories"][df_ken_rest["categories"].str.contains('Dominican|Dominicano')].count()

0

In [27]:
kennesaw_map = folium.Map(location=[kennesaw['lat'], kennesaw['lng']], zoom_start=13) # generate map centred around Ecco

# Add restaurants to the map
for lat, lng, label in zip(df_ken_rest.lat, df_ken_rest.lng, df_ken_rest.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='yellow',
        fill_opacity=0.6
        ).add_to(kennesaw_map)

# display map
kennesaw_map

<hr size=30px></hr>

# Capstone Project - The Battle of Neighborhoods (Week 2)

### 3- Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

### 4- Results section where you discuss the results.

### 5- Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

### 6- Conclusion section where you conclude the report.